# Team12 Web-scraping

In [ ]:
#import the environments for web-scaping
import requests
from bs4 import BeautifulSoup
import collections
import numpy as np
import pandas as pd
from google.colab import drive

##Scrape Job Post Links

In [ ]:
default='https://www.linkedin.com/jobs/search/'
#keywords='?keywords=software%20engineer'
#keywords='?keywords=data%20scientist'
#keywords='?keywords=product%20manager'
#keywords='?keywords=qa%20testing'
keywords='?keywords=ui%20designer'
job_type=default+keywords
job_search_links=[]
for i in range(0,1501,25):
    page='&start=%s' % (i,)
    if i==0:
        job_search_links.append(job_type)
    else:
        job_search_links.append(job_type+page)

In [ ]:
links=[]
for js_link in job_search_links:
    js_response=requests.get(js_link)
    js_soup=BeautifulSoup(js_response.content,'html.parser')
    for k in js_soup.find_all(class_='base-card__full-link'):
        links.append(k['href'])
links[:5]

['https://www.linkedin.com/jobs/view/visual-ui-designer-at-amazon-3005205774?refId=54QE4MqoXqfvyhVnwDx8jA%3D%3D&trackingId=QTba4RxHmIuraNfeA5UeMw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://www.linkedin.com/jobs/view/user-interface-designer-at-portfolio-creative-3002451356?refId=54QE4MqoXqfvyhVnwDx8jA%3D%3D&trackingId=pJtIUI6D91phu0Mh6tubZw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://www.linkedin.com/jobs/view/remote-user-interface-designer-at-adecco-3007586477?refId=54QE4MqoXqfvyhVnwDx8jA%3D%3D&trackingId=7nwEezEHxG%2FZU7F1%2FUId%2BA%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://www.linkedin.com/jobs/view/ux-visual-designer-at-u-s-bank-3007561485?refId=54QE4MqoXqfvyhVnwDx8jA%3D%3D&trackingId=cbetET3CT3r6xKzX%2BGcr0A%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://www.linkedin.com/jobs/view/ui-designer-at-cygnus-professionals-inc-3006407249?refId=54Q

##Scrape Job Detials From Job Post

In [ ]:
#catch company name,job title and location information
def getNameLocation(soup):
    company,name,location='','',''
    for show in soup.find_all('title'):
        try:
            head=show.text
            head=head.split()
            company=head[0]
            name_b=head.index('hiring')
            name_e=head.index('in')
            name=' '.join(head[name_b+1:name_e])
            location_b=head.index('in')
            location_e=head.index('|')
            location=' '.join(head[location_b+1:location_e])
        except:
            company,name,location=None,None,None
    return company,name,location

In [ ]:
#catch job description
def getDescription(soup):
    for i,show in enumerate(soup.find_all(class_='description')):
        return ' '.join(show.text.split())

In [ ]:
#scrap information from LinkedIn and store the information in a dict
info=collections.defaultdict(list)
for link in links:
    response=requests.get(link)
    soup=BeautifulSoup(response.content,'html.parser')
    company,name,location=getNameLocation(soup)
    description=getDescription(soup)
    info['company'].append(company)
    info['job_title'].append(name)
    info['location'].append(location)
    info['description'].append(description)

In [ ]:
#transform the dict to a DataFrame
df=pd.DataFrame(info)
df.dropna(how='any')
#df.head(10)

,company,job_title,location,description
0,Amazon,Visual UI Designer,"New York, New York, United States",Job SummaryDESCRIPTIONThe AWS Analytics team d...
1,Portfolio,User Interface Designer,"Columbus, Ohio Metropolitan Area",Direct message the job poster from Portfolio C...
2,Adecco,Remote User Interface Designer,"Maryland, United States",Direct message the job poster from Adecco Shlo...
3,U.S.,UX Visual Designer,"Atlanta, Georgia, United States","At U.S. Bank, we're passionate about helping c..."
4,Cygnus,UI Designer,"Miami, Florida, United States",• Ability to translate conceptual ideas into c...
...,...,...,...,...
1509,Truist,Lightstream Graphic and UI Designer,"San Diego, California, United States",The position is described below. If you want t...
1510,U.S.,UX Visual Designer,"San Francisco, California, United States","At U.S. Bank, we're passionate about helping c..."
1516,Practivest,UI Designer (Remote),"Bethesda, Maryland, United States",Are you a creative person looking to boost you...
1518,Activision,UI Designer,"Los Angeles, California, United States",Job TitleUI DesignerRequisition IDR013542Job D...


In [ ]:
len(df)

1524

In [ ]:
drive.mount('/drive')
#title='software_engineer'
#title='data_scientist'
#title='product_manager'
#title='QA_test_engineer'
title='UI_UX_designer'
location= '/drive/My Drive/Data-X Project/' + title + '.csv'
df.to_csv(location)

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
